In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open("..data/bi_grams.txt",encoding="utf-8") as f:
    bi_gramtxt=f.read()
bi_gram_list=[]
for each in bi_gramtxt.split(","):
    each=each[:-1]
    each=each[2:]
    each=each.replace(" ","_")
    bi_gram_list.append(each)

In [19]:
class FTokenizer():
    def __init__(self,bi_gram_list):
        self.bi_gram_list=bi_gram_list

    def bi_gram_checker(self,bi_gram_word):
        for i in self.bi_gram_list:
            if (bi_gram_word==i):
                return True
        return False
    
    def fit_on_text(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (myContinue):
                    myContinue=False
                    continue
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        myContinue=True
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (myContinue==False):  
                    if (tokens.count(listtokens[i])==0):
                        tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word

    def fit_on_text2(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (tokens.count(listtokens[i])==0 and listtokens[i]!=""):
                    tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word
    
    def get_word_to_index(self):
        return self.word_to_index
    
    def get_index_to_word(self):
        return self.index_to_word

    def get_max_length(self,data):
        maxsen=0
        for i in data:
            if maxsen<len(i):
                maxsen=len(i)
        return maxsen

    def text_to_sequence(self,textdata):
        d_continue=False
        output=[]
        for text in textdata:
            text=text.lower().split(" ")
            text.append("")
            sth=[]
            d_continue=False
            for i in range(0,len(text)-1):
                if d_continue==True:
                    d_continue=False
                    continue
                if (text[i+1]!=""):
                    pairtext=text[i]+"_"+text[i+1]
                    for j in self.word_to_index:
                        if pairtext.lower()==j:
                            sth.append(self.word_to_index[j])
                            d_continue=True
                            break
                if d_continue==False:
                    for count,j in enumerate(self.word_to_index):
                        if text[i].lower()==j:
                            sth.append(self.word_to_index[j])
                            break
                        if count==len(self.word_to_index)-1:
                            sth.append(1)
            output.append(sth)
        return output
    
    def intseq_to_index(self,sequence):
        output=[]
        for i in sequence:
            output.append(self.index_to_word[i])
        return output

    def pad_sequence(self,datasequence,maxlen):
        for i in datasequence:
            while len(i)!=maxlen:
                i.insert(0,0)
        return datasequence

    def bi_gram_ize(self,text):
        listtokens=text.lower().split(" ")
        listtokens.append("")
        myContinue=False
        tokens=[]
        for i in range(0,len(listtokens)-1):
            if (myContinue):
                myContinue=False
                continue
            if (listtokens[i+1]!=""):
                pairtext=listtokens[i]+"_"+listtokens[i+1]
                if (self.bi_gram_checker(pairtext)):
                    myContinue=True
                    if (tokens.count(pairtext)==0):
                        tokens.append(pairtext)
            if (myContinue==False):  
                if (tokens.count(listtokens[i])==0):
                    tokens.append(listtokens[i])
        return tokens


In [14]:
Tokenizer=FTokenizer(bi_gram_list=bi_gram_list)
Tokenizer.fit_on_text2(["Hôm nay tôi đi học","Hôm nay tôi đi chơi","Bạn tới chơi nhà","Có sao không vậy","Tôi thích ăn sandwich","Chơi minecraft đi","Hôm nay tôi chưa làm xong bài tập quan trọng nữa"])
te=Tokenizer.text_to_sequence(["Hôm nay tôi đi học","Hôm nay tôi đi chơi","Bạn tới chơi nhà","Có sao không vậy","Tôi thích ăn sandwich","Chơi minecraft đi","Hôm nay tôi chưa làm xong bài tập quan trọng nữa"])
maxlen=Tokenizer.get_max_length(te)
output=Tokenizer.pad_sequence(te,maxlen)
Tokenizer.word_to_index


{'<PAD>': 0,
 '<OOV>': 1,
 'hôm_nay': 2,
 'hôm': 3,
 'nay': 4,
 'tôi': 5,
 'đi': 6,
 'học': 7,
 'chơi': 8,
 'bạn': 9,
 'tới': 10,
 'nhà': 11,
 'có': 12,
 'sao': 13,
 'không': 14,
 'vậy': 15,
 'thích': 16,
 'ăn': 17,
 'sandwich': 18,
 'minecraft': 19,
 'chưa': 20,
 'làm': 21,
 'xong': 22,
 'bài_tập': 23,
 'bài': 24,
 'tập': 25,
 'quan_trọng': 26,
 'quan': 27,
 'trọng': 28,
 'nữa': 29}

In [21]:
print(Tokenizer.bi_gram_ize("Buổi họp hôm nay rất quan trọng"))

['buổi', 'họp', 'hôm_nay', 'rất', 'quan_trọng']
